In [38]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
import warnings
warnings.filterwarnings("ignore") 

In [39]:
base_dir = os.path.dirname(os.path.abspath("..\data"))
path = os.path.join(base_dir, r"data")
os.chdir(path) 

In [40]:
df = pd.read_csv("Cleaned data\data.csv")

In [41]:
# drop columns
df = df.drop(columns=[
    "DATE","MONTHDATE","FORECASTDEMAND","MIN","ACCMIN","MAX","ACCMAX","RAINPERIOD","QUANTITY","QUANTITYMONTHCUM","QUANTITYTOTALCUM","HUMIDITYMIN","HUMIDITYMAX", "RAIN", "WEEKEND"
], errors="ignore")
df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,TOTALDEMAND,HOLIDAY,SOLAR,RRP,OUTPUT,WEEKDAY,TEMPAVE
0,2016,1,1,0.656341,15.902439,6853.633437,1.0,32.2,38.472917,46.93,4,21.95
1,2016,1,2,0.656341,15.902439,6727.613958,0.0,21.7,36.907292,46.93,5,21.00
2,2016,1,3,0.688837,14.488372,6616.406076,0.0,10.3,31.997083,46.93,6,21.55
3,2016,1,4,0.679545,22.477273,7367.750278,0.0,6.4,33.424583,46.93,0,20.90
4,2016,1,5,0.768837,22.581395,7462.242014,0.0,4.4,33.053958,46.93,1,19.05


In [42]:
#one hot encoding weekday
def one_hot_encode(df, col, names):
    array = np.array(df[col])
    integer_encoded = array.reshape(len(array), 1)
    onehotencoder = OneHotEncoder()
    onehot_encoded = onehotencoder.fit_transform(integer_encoded).toarray()
    df_encoded = pd.DataFrame(onehot_encoded, columns=names)
    df = pd.concat([df, df_encoded], axis=1)
    df = df.drop(columns=[col], errors="ignore")
    return df

df = one_hot_encode(df, col="WEEKDAY", names=['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'])
df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,TOTALDEMAND,HOLIDAY,SOLAR,RRP,OUTPUT,TEMPAVE,MON,TUE,WED,THU,FRI,SAT,SUN
0,2016,1,1,0.656341,15.902439,6853.633437,1.0,32.2,38.472917,46.93,21.95,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016,1,2,0.656341,15.902439,6727.613958,0.0,21.7,36.907292,46.93,21.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016,1,3,0.688837,14.488372,6616.406076,0.0,10.3,31.997083,46.93,21.55,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2016,1,4,0.679545,22.477273,7367.750278,0.0,6.4,33.424583,46.93,20.90,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,1,5,0.768837,22.581395,7462.242014,0.0,4.4,33.053958,46.93,19.05,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# identify response variable and predictors
X = df.drop(['TOTALDEMAND'], axis=1)
y = df['TOTALDEMAND'] 

In [44]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
# define scaler and transform data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [47]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [48]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [56]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_reshaped, y_test))

Epoch 1/100
50/50 [==============================] - 1s 13ms/step - loss: 59736516.0000 - val_loss: 59601388.0000
Epoch 2/100
50/50 [==============================] - 1s 12ms/step - loss: 59699824.0000 - val_loss: 59564736.0000
Epoch 3/100
50/50 [==============================] - 1s 12ms/step - loss: 59663160.0000 - val_loss: 59528084.0000
Epoch 4/100
50/50 [==============================] - 1s 12ms/step - loss: 59626492.0000 - val_loss: 59491460.0000
Epoch 5/100
50/50 [==============================] - 1s 12ms/step - loss: 59589860.0000 - val_loss: 59454804.0000
Epoch 6/100
50/50 [==============================] - 1s 13ms/step - loss: 59553232.0000 - val_loss: 59418196.0000
Epoch 7/100
50/50 [==============================] - 1s 12ms/step - loss: 59516600.0000 - val_loss: 59381580.0000
Epoch 8/100
50/50 [==============================] - 1s 12ms/step - loss: 59479992.0000 - val_loss: 59345016.0000
Epoch 9/100
50/50 [==============================] - 1s 11ms/step - loss: 59443396.0000 

In [62]:
# Make predictions
y_pred = model.predict(X_test_reshaped)

13/13 [==============================] - 0s 5ms/step
